In [2]:
import numpy as np
import peewee as pw
import pandas as pd

from tqdm import tqdm
from structure import Const, Solve, database_connection
from solver import CONST_DEFAULT,DIR_PATH,df_from_comsol,write

In [2]:
df = df_from_comsol()
df.head()

,Ke,KH,Kr,Kdisp,KqH,Ks,Kd,Kc,Kp,KrD,...,Q,DH,QHD,QHH,prod,Qm,DHp,QH,D,QD
0,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0
1,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,2.000000e-12,0.0,0.0,0.0,0.0,0.0
2,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,4.000000e-12,0.0,0.0,0.0,0.0,0.0
3,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,8.000000e-12,0.0,0.0,0.0,0.0,0.0
4,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,1.600000e-11,0.0,0.0,0.0,0.0,0.0


In [ ]:
# def get(cond:list):
#     with database_connection:
#         q= Solve.select(Solve).where(Solve.name=='test_#2').get()
#     name = q.name    
#     df= pd.read_json(q.data)
#     return name

In [17]:
with database_connection:
    q= Solve.select(Solve).where(Solve.name=='test_#2').get()
    q2 = Const.select(Const).join(Solve).where(Const.solve_id==q).execute()
df= pd.read_json(q.data)
for i in q2:
    df[i.name]=i.value
df

,Time,Q,DH,QHD,QHH,prod,Qm,DHp,QH,D,...,Kr,Kdisp,KqH,Ks,Kd,Kc,Kp,KrD,Kph,light
0,0.000000e+00,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,...,1000000000.00000,1000000000.00000,2000.00000,3.00000,0.05000,1.00000,0.00100,1000000000.00000,0.00001,0.00002
1,8.000000e-10,0.999984,0.999984,0.000000e+00,0.000000e+00,0.000000,0.000016,0.000016,2.000000e-10,2.000000e-10,...,1000000000.00000,1000000000.00000,2000.00000,3.00000,0.05000,1.00000,0.00100,1000000000.00000,0.00001,0.00002
2,1.400000e-09,0.999973,0.999973,2.300638e-19,2.300635e-19,0.000000,0.000027,0.000027,6.000000e-10,6.000000e-10,...,1000000000.00000,1000000000.00000,2000.00000,3.00000,0.05000,1.00000,0.00100,1000000000.00000,0.00001,0.00002
3,2.500000e-09,0.999949,0.999949,1.568128e-17,1.568125e-17,0.000000,0.000051,0.000051,3.600000e-09,3.600000e-09,...,1000000000.00000,1000000000.00000,2000.00000,3.00000,0.05000,1.00000,0.00100,1000000000.00000,0.00001,0.00002
4,4.900000e-09,0.999903,0.999903,0.000000e+00,0.000000e+00,0.000000,0.000097,0.000097,2.590000e-08,2.590000e-08,...,1000000000.00000,1000000000.00000,2000.00000,3.00000,0.05000,1.00000,0.00100,1000000000.00000,0.00001,0.00002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,1.916527e-04,0.403306,0.115557,3.071314e-01,2.878140e-01,0.287875,0.000968,0.000968,7.157758e-04,5.294404e-04,...,1000000000.00000,1000000000.00000,2000.00000,3.00000,0.05000,1.00000,0.00100,1000000000.00000,0.00001,0.00002
360,1.938447e-04,0.401890,0.113535,3.079539e-01,2.884213e-01,0.288482,0.000958,0.000958,7.110558e-04,5.233532e-04,...,1000000000.00000,1000000000.00000,2000.00000,3.00000,0.05000,1.00000,0.00100,1000000000.00000,0.00001,0.00002
361,1.960367e-04,0.400503,0.111555,3.087616e-01,2.890146e-01,0.289076,0.000948,0.000948,7.064280e-04,5.173618e-04,...,1000000000.00000,1000000000.00000,2000.00000,3.00000,0.05000,1.00000,0.00100,1000000000.00000,0.00001,0.00002
362,1.982287e-04,0.399143,0.109616,3.095549e-01,2.895945e-01,0.289656,0.000938,0.000938,7.018904e-04,5.114637e-04,...,1000000000.00000,1000000000.00000,2000.00000,3.00000,0.05000,1.00000,0.00100,1000000000.00000,0.00001,0.00002


In [ ]:
a = {'light':()}

In [48]:
with database_connection:
    q = (Solve
         .select(Solve)
         .join(Const)
         .where((Const.name == 'light'), (Const.value < 0.00003))
        ).execute() # yapf: disable
    print(list(q))

[<Solve: 1>, <Solve: 2>, <Solve: 3>]


In [40]:
with database_connection:
    q = (Const
         .select(Const)
         .join(Solve)
         .where(Const.name == 'light'and Const.value < 0.00003)).get() # yapf: disable
    for i in q:
        print(i.name)

TypeError: 'Const' object is not iterable

In [ ]:
# with database_connection:
#     for :
#         q= Solve.select(Solve).where(Solve.name=='test_#2').get()
#         q2 = Const.select(Const).join(Solve).where(Const.solve_id==q).execute()
#         df= pd.read_json(q.data)
#         for i in q2:
#             df[i.name]=i.value

# procedure

In [9]:
with database_connection:
    res =database_connection.execute_sql('call 	show_consts();')
    # res2 = database_connection.execute_sql( list(res)[0][0])
    # q= Const.select(Const).join( res2)
    

for i in res:
    print(i)
        

('SELECT solve_id, \nSUM( CASE WHEN name = "Kc" THEN value ELSE 0 END ) AS Kc,\nSUM( CASE WHEN name = "Kd" THEN value ELSE 0 END ) AS Kd,\nSUM( CASE WHEN name = "Kdisp" THEN value ELSE 0 END ) AS Kdisp,\nSUM( CASE WHEN name = "Ke" THEN value ELSE 0 END ) AS Ke,\nSUM( CASE WHEN name = "KH" THEN value ELSE 0 END ) AS KH,\nSUM( CASE WHEN name = "Kp" THEN value ELSE 0 END ) AS Kp,\nSUM( CASE WHEN name = "Kph" THEN value ELSE 0 END ) AS Kph,\nSUM( CASE WHEN name = "KqH" THEN value ELSE 0 END ) AS KqH,\nSUM( CASE WHEN name = "Kr" THEN value ELSE 0 END ) AS Kr,\nSUM( CASE WHEN name = "KrD" THEN value ELSE 0 END ) AS KrD,\nSUM( CASE WHEN name = "Ks" THEN value ELSE 0 END ) AS Ks,\nSUM( CASE WHEN name = "light" THEN value ELSE 0 END ) AS light\nFROM const \nGROUP BY solve_id',)


In [ ]:
# CREATE PROCEDURE test()
SET @sql =NULL;
SELECT
GROUP_CONCAT(
  	 DISTINCT CONCAT(
      'SUM(
      CASE WHEN name = "',
      name,
      '" THEN value ELSE 0 END) AS ',
      name)
)
INTO @sql
FROM const;

SET @sql = CONCAT('SELECT solve_id, ', @sql,
  ' FROM const GROUP BY solve_id');
SELECT @sql;

PREPARE stmt FROM @sql;
EXECUTE stmt; 
